<a href="https://colab.research.google.com/github/crodier1/Age_Calculator/blob/master/Absenteeism_at_work_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
dataset = pd.read_csv('Absenteeism_at_work.csv')

dataset.columns = [c.strip() for c in dataset.columns]

y = dataset['Absenteeism time in hours']

x = dataset.drop(['ID', 'Absenteeism time in hours'], axis=1)

In [ ]:
def set_absentee_categories(hours):
  if hours == 0 :
    return 0
  if hours > 0 and hours <= 8:
    return 1
  if hours > 8 and hours <= 16:
    return 2
  if hours > 16 and hours <= 24:
    return 3
  if hours > 24 and hours <= 32:
    return 4
  if hours > 32 and hours <= 40:
    return 5
  if hours > 40 and hours <= 48:
    return 6
  if hours > 48 and hours <= 56:
    return 7
  if hours > 56 and hours <= 64:
    return 8
  if hours > 64:
    return 9

y = y.apply(lambda h : set_absentee_categories(h))

In [ ]:
def get_dummies(colName):

  dummies = pd.get_dummies(x[colName])

  cols = dummies.columns

  x[[f"{colName}-{n}" for n in cols]] = dummies[cols]

  x.drop(colName, axis=1, inplace=True)

In [ ]:
categories = ['Reason for absence', 'Month of absence', 'Day of the week', 'Seasons', 'Disciplinary failure', 'Education', 'Social drinker', 'Social smoker']

for c in categories:
  get_dummies(c)



In [ ]:
x['Work load Average'] = x['Work load Average/day'].str.slice(0,3).astype(int)

x['day'] = x['Work load Average/day'].str.slice(4).astype(int)

x.drop('Work load Average/day', inplace=True, axis=1)

In [ ]:
all_variables = x
all_variables['Absenteeism time in hours'] = y

for c in x.columns:
  print(all_variables[[c, 'Absenteeism time in hours']].corr())

                           Transportation expense  Absenteeism time in hours
Transportation expense                   1.000000                  -0.032691
Absenteeism time in hours               -0.032691                   1.000000
                                 Distance from Residence to Work  \
Distance from Residence to Work                         1.000000   
Absenteeism time in hours                              -0.079993   

                                 Absenteeism time in hours  
Distance from Residence to Work                  -0.079993  
Absenteeism time in hours                         1.000000  
                           Service time  Absenteeism time in hours
Service time                   1.000000                   0.030595
Absenteeism time in hours      0.030595                   1.000000
                                Age  Absenteeism time in hours
Age                        1.000000                   0.042326
Absenteeism time in hours  0.042326                   

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train) 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [ ]:
model = Sequential()

model.add(Dense(X_train_scaled.shape[1], activation='relu'))

model.add(Dense(3000, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(tf.keras.layers.Dropout(.4))
model.add(Dense(10, activation='sigmoid')) 

In [ ]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.01)

model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='SGD')

model.fit(X_train_scaled, y_train, epochs=10, batch_size=25, verbose=1, validation_data=(X_test, y_test))

Epoch 1/10
20/20 [==============================] - 3s 88ms/step - loss: 1.4512 - accuracy: 0.7697 - val_loss: 31.3351 - val_accuracy: 0.8694
Epoch 2/10
20/20 [==============================] - 2s 75ms/step - loss: 0.8498 - accuracy: 0.8485 - val_loss: 38.5054 - val_accuracy: 0.8694
Epoch 3/10
20/20 [==============================] - 1s 74ms/step - loss: 0.7538 - accuracy: 0.8485 - val_loss: 40.3375 - val_accuracy: 0.8694
Epoch 4/10
20/20 [==============================] - 2s 92ms/step - loss: 0.6824 - accuracy: 0.8485 - val_loss: 41.1430 - val_accuracy: 0.8694
Epoch 5/10
20/20 [==============================] - 2s 107ms/step - loss: 0.6377 - accuracy: 0.8485 - val_loss: 41.0142 - val_accuracy: 0.8694
Epoch 6/10
20/20 [==============================] - 2s 86ms/step - loss: 0.5899 - accuracy: 0.8505 - val_loss: 41.4309 - val_accuracy: 0.8694
Epoch 7/10
20/20 [==============================] - 2s 85ms/step - loss: 0.5560 - accuracy: 0.8525 - val_loss: 40.8446 - val_accuracy: 0.8694
Epoch